In [47]:
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Створюємо спільний датасет:

In [48]:
data_folder = '/content/drive/MyDrive/data'

all_data = pd.DataFrame()

for activity_folder in os.listdir(data_folder):
    activity_path = os.path.join(data_folder, activity_folder)
    if not os.path.isdir(activity_path):
        continue

    for file_name in os.listdir(activity_path):
        file_path = os.path.join(activity_path, file_name)
        if not file_name.endswith('.csv'):
            continue

        df = pd.read_csv(file_path)

        df['activity'] = activity_folder

        all_data = pd.concat([all_data, df], ignore_index=True)

all_data = all_data.sample(frac=1).reset_index(drop=True)

X = all_data.drop('activity', axis=1)
y = all_data['activity']

Робимо тренувальні та тестові данні:

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Часові ознаки:

In [50]:
X_train['time_mean'] = X_train.mean(axis=1)
X_test['time_mean'] = X_test.mean(axis=1)
X_train['time_min'] = X_train.min(axis=1)
X_test['time_min'] = X_test.min(axis=1)
X_train['time_max'] = X_train.max(axis=1)
X_test['time_max'] = X_test.max(axis=1)
X_train['time_std'] = X_train.std(axis=1)
X_test['time_std'] = X_test.std(axis=1)

Класифікації:

In [51]:
svm_model = SVC()
rf_model = RandomForestClassifier()

Тренування моделей:

In [52]:
svm_model.fit(X_train, y_train)

SVC()

In [53]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

Прогнозування на тестових даних:

In [54]:
svm_pred = svm_model.predict(X_test)

In [55]:
rf_pred = rf_model.predict(X_test)

Оцінка результатів:

In [56]:
svm_accuracy = accuracy_score(y_test, svm_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

print("Accuracy (SVM):", svm_accuracy)
print("Accuracy (Random Forest):", rf_accuracy)

Accuracy (SVM): 0.896405430422709
Accuracy (Random Forest): 0.9997943021701121


In [57]:
svm_report = classification_report(y_test, svm_pred)
print("SVM Results:")
print(svm_report)

rf_report = classification_report(y_test, rf_pred)
print("Random Forest Results:")
print(rf_report)

SVM Results:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      6232
     running       0.93      0.91      0.92     20443
      stairs       1.00      0.00      0.01      1004
     walking       0.81      0.91      0.85     11213

    accuracy                           0.90     38892
   macro avg       0.93      0.70      0.69     38892
weighted avg       0.90      0.90      0.89     38892

Random Forest Results:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6232
     running       1.00      1.00      1.00     20443
      stairs       1.00      0.99      1.00      1004
     walking       1.00      1.00      1.00     11213

    accuracy                           1.00     38892
   macro avg       1.00      1.00      1.00     38892
weighted avg       1.00      1.00      1.00     38892



Висновок: Виходячи з отриманих таймінгів розрахунків, тренування SVC моделі зайняло 8 хвилин, в той час, як час тренування Random Forest ~25 секунд. Прогнозування тестових даних для SVC моделі зайняло 1 хвилину, а для Random Forest - меньше секунди. Також, слід зазначити, що згідно метрик, в данному випадку, Random Forest дає більш точніші результати прогнозування.